In [18]:
using DataFrames
using LinearAlgebra
using Distributions
using CSV
using NPZ
using Base.Threads
using BenchmarkTools
using ProgressMeter
using Plots
using Random
using Pkg
using StatsPlots

In [20]:
Pkg.develop(path="c:\\Users\\guibi\\.julia\\dev\\MyARCHModels")
using MyARCHModels

   Resolving package versions...
  No Changes to `C:\Users\guibi\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\guibi\.julia\environments\v1.10\Manifest.toml`


In [21]:
data100 = CSV.read("returns_data100.csv", DataFrame)[2:end,2:end]
data100_mat = coalesce.(Matrix(data100), 0.0)

data500 = CSV.read("returns_data500.csv", DataFrame)[2:end,2:end]
data500_mat = coalesce.(Matrix(data500), 0.0)

data1000 = CSV.read("returns_data1000.csv", DataFrame)[2:end,2:end]
data1000_mat = coalesce.(Matrix(data1000), 0.0)

data750_long = CSV.read("returns_data_long_750.csv", DataFrame)[2:end,2:end]
data750_long_mat = coalesce.(Matrix(data750_long), 0.0)

data100_long = CSV.read("returns_data_long_100.csv", DataFrame)[2:end,2:end]
data100_long_mat = coalesce.(Matrix(data100_long), 0.0)


2430×100 Matrix{Float64}:
 -0.0197005   -0.0340713    -0.00938265   …  -0.0210244   -0.0142187
  0.0247473    0.00609711    0.029517         0.0019837    0.0319436
 -0.00117562   0.0150837     0.01034          0.020595     0.00406034
 -0.0145532   -0.0293693    -0.00400012      -0.0126992   -0.00628216
 -0.00896075  -0.0197204    -0.00968031      -0.0144571   -0.00464423
 -0.00448221  -0.0158481    -0.00124603   …   0.0323843    6.2074e-5
 -0.00774711  -0.0247992    -0.0168153       -0.0293966   -0.00292146
 -0.0166205   -0.0187335    -0.0165383       -0.00818776  -0.00599351
  0.00347586   0.00885941    0.00930165       0.00639618   0.000375601
 -0.0108199   -0.0370619    -0.000399216     -0.00359282   0.0016887
  0.00769795  -0.0034384     0.000957764  …   0.00955238   0.0167354
  0.0076983    0.0536441     0.0129977        0.0203888    0.00862939
 -0.012132    -0.0197804    -0.016914         0.0         -0.0144239
  ⋮                                       ⋱               
  0.035567

In [4]:
function markowitz_min_var_port(Σ::Matrix{Float64})
    (n, _) = size(Σ)
    SigmaI = inv(Σ)
    e = ones(n)
    den = e' * SigmaI * e
    return (SigmaI * e) / den
end

function markowitz_min_var(Σ::Matrix{Float64})
    w = markowitz_min_var_port(Σ)
    return w' * Σ * w
end

function get_predictions(data::Matrix{Float64})
    (n,d) = size(data)
    pred_cov = [Matrix{Float64}(I(d)) for _ in 1:n]
    @showprogress for time in 12:n
            sliced_data = deepcopy(data[1:time,:])
            spec = DCC{1,1}
            res = fit(spec, sliced_data)
            pred_cov[time] = predict(res)
    end
    return pred_cov
end

function loss(Sigma::Matrix{Float64}, Sigma_hat::Matrix{Float64})
    (n, _) = size(Sigma)
    Sigma_hat_I = inv(Sigma_hat)
    a = tr(Sigma_hat_I * Sigma * Sigma_hat_I) / n
    b = (tr(Sigma_hat_I) /n)^2
    c = 1 / ( tr(inv(Sigma)) / n )
    return a / b - c
end


function analytical_shrinkage(X)
n, p = size(X)
@assert n >= 12 # important: sample size n must be >= 12
sample = Symmetric(X'*X) / n
E = eigen(sample)
lambda = E.values
u = E.vectors

# compute analytical nonlinear shrinkage kernel formula
lambda = lambda[max(1, p-n+1):p]
L = repeat(lambda, 1, min(p, n))
h = n^(-1/3) # Equation (4.9)
H = h*L'
x = (L-L') ./ H
ftilde = (3/4/sqrt(5)) * mean(max.(1 .- x.^2 ./ 5, 0) ./ H, dims=2) # Equation (4.7)
Hftemp = (-3/10/pi) * x + (3/4/sqrt(5)/pi) * (1 .- x.^2 ./ 5) .* log.(abs.((sqrt(5).-x) ./ (sqrt(5).+x))) # Equation (4.8)
Hftemp[abs.(x) .== sqrt(5)] .= (-3/10/pi) .* x[abs.(x) .== sqrt(5)]
Hftilde = mean(Hftemp./H, dims=2)
if p<=n
    dtilde = lambda ./ ((pi * (p/n) *lambda .* ftilde).^2  + (1 .- (p/n) .- pi * (p/n) * lambda .* Hftilde).^2) # Equation (4.3)
else
    Hftilde0 = (1/pi) * (3/10/h^2 + 3/4/sqrt(5)/h*(1-1/5/h^2) * log((1+sqrt(5)*h)/(1-sqrt(5)*h)))*mean(1 ./ lambda) # Equation (C.8)
    dtilde0 = 1 / (pi * (p-n) / n * Hftilde0) # Equation (C.5)
    dtilde1 = lambda ./ (pi^2*lambda.^2 .* (ftilde.^2 + Hftilde.^2)) # Eq. (C.4)
    dtilde = [dtilde0*ones(p-n,1); dtilde1]
end
return u * Diagonal(dtilde[:]) * u'
end

function frob(A)
    (p, _) = size(A)
    return sqrt(tr(A * A') / p)
end

function linear_shrinkage(X)
    n, p = size(X)
    sample = Symmetric(X'*X) / n
    E = eigen(sample)
    lambda = E.values
    u = E.vectors
    
    m = tr(sample) / p
    d = frob(sample - I * m)^2

    b_bar = sum([frob(X[k, :] * X[k, :]' - sample)^2 for k in 1:n]) / n^2

    b = min(b_bar, d)

    a = d - b

    shrunk = (b/d)* m * I + (a / d) * sample

    return shrunk
end

linear_shrinkage (generic function with 1 method)

### Monte Carlo 500

In [5]:
mod500 = fit(DCC{1,1}, data500_mat)
p = 500
R500 = mod500.spec.R;
real_covs500 = covariances(mod500)
simulation_spec500 = DCC{1, 1}(R500, [.96, .02], [GARCH{1, 1}([10e-5, .9, .05]) for _ in 1:p])

┌ Warning: Failed to achieve finite new evaluation point, using alpha=0
└ @ LineSearches C:\Users\guibi\.julia\packages\LineSearches\jgnxK\src\hagerzhang.jl:155
┌ Warning: Failed to achieve finite new evaluation point, using alpha=0
└ @ LineSearches C:\Users\guibi\.julia\packages\LineSearches\jgnxK\src\hagerzhang.jl:155


DCC{1, 1, GARCH{1, 1}} specification.

───────────────────────
               β₁    α₁
───────────────────────
Parameters:  0.96  0.02
───────────────────────


In [6]:
linear_loss_sim500 = [0.0 for i in 1:100]

@showprogress for i in 1:100
    sim_data500 = simulate(simulation_spec500, 1250; warmup = 100).data
    est500 = fit(DCC{1,1}, sim_data500, method = :linear)
    est_covs500 = covariances(est500)
    linear_loss_sim500[i] = mean([loss(real, est) for (real,est) in zip(real_covs500, est_covs500)])
end

npzwrite("montecarlo_new//linear_loss_sim500.npy", linear_loss_sim500)

Progress: 100%|█████████████████████████████████████████| Time: 2:03:18


In [7]:
analytical_loss_sim500 = [0.0 for i in 1:100]

@showprogress for i in 1:100
    sim_data500 = simulate(simulation_spec500, 1250; warmup = 100).data
    est500 = fit(DCC{1,1}, sim_data500, method = :largescale)
    est_covs500 = covariances(est500)
    analytical_loss_sim500[i] = mean([loss(real, est) for (real,est) in zip(real_covs500, est_covs500)])
end

npzwrite("montecarlo_new//anal_loss_sim500.npy", analytical_loss_sim500)

Progress: 100%|█████████████████████████████████████████| Time: 1:59:37


In [8]:
sample_cov_sim500 = [0.0 for i in 1:100]

@showprogress for i in 1:100
    sim_data500 = simulate(simulation_spec500, 1250; warmup = 100).data
    est500 = fit(DCC{1,1}, sim_data500, method = :twostep)
    est_covs500 = covariances(est500)
    sample_cov_sim500[i] = mean([loss(real, est) for (real,est) in zip(real_covs500, est_covs500)])
end

npzwrite("montecarlo_new//cov_loss_sim500.npy", sample_cov_sim500)

Progress: 100%|█████████████████████████████████████████| Time: 1:59:42


### Monte Carlo 100

In [10]:
mod100 = fit(DCC{1,1}, data100_mat)
p = 100
R100 = mod100.spec.R;
real_covs100 = covariances(mod100)
simulation_spec100 = DCC{1, 1}(R100, [.96, .02], [GARCH{1, 1}([10e-5, .9, .05]) for _ in 1:p])

DCC{1, 1, GARCH{1, 1}} specification.

───────────────────────
               β₁    α₁
───────────────────────
Parameters:  0.96  0.02
───────────────────────


In [11]:
linear_loss_sim100 = [0.0 for i in 1:100]

@showprogress for i in 1:100
    sim_data100 = simulate(simulation_spec100, 1250; warmup = 100).data
    est_model100 = fit(DCC{1,1}, sim_data100, method = :linear)
    est_covs100 = covariances(est_model100)
    linear_loss_sim100[i] = mean([loss(real, est) for (real,est) in zip(real_covs100, est_covs100)])
end

npzwrite("montecarlo_new//linear_loss_sim100.npy", linear_loss_sim100)

Progress: 100%|█████████████████████████████████████████| Time: 0:09:11


In [12]:
analytical_loss_sim100 = [0.0 for i in 1:100]

@showprogress for i in 1:100
    sim_data100 = simulate(simulation_spec100, 1000; warmup = 100).data
    est_model100 = fit(DCC{1,1}, sim_data100, method = :largescale)
    est_covs100 = covariances(est_model100)
    analytical_loss_sim100[i] = mean([loss(real, est) for (real,est) in zip(real_covs100, est_covs100)])
end

npzwrite("montecarlo_new//anal_loss_sim100.npy", analytical_loss_sim100)

Progress: 100%|█████████████████████████████████████████| Time: 0:07:20


In [13]:
sample_cov_sim100 = [0.0 for i in 1:100]

@showprogress for i in 1:100
    sim_data100 = simulate(simulation_spec100, 1250; warmup = 100).data
    est_model100 = fit(DCC{1,1}, sim_data100, method = :twostep)
    est_covs100 = covariances(est_model100)
    sample_cov_sim100[i] = mean([loss(real, est) for (real,est) in zip(real_covs100, est_covs100)])
end

npzwrite("montecarlo_new//sample_loss_sim100.npy", sample_cov_sim100)

Progress: 100%|█████████████████████████████████████████| Time: 0:09:29


### Monte Carlo 750

In [5]:
mod750 = fit(DCC{1,1}, data750_mat)
p = 750
R750 = mod750.spec.R;
real_covs750 = covariances(mod750)
simulation_spec750 = DCC{1, 1}(R750, [.96, .02], [GARCH{1, 1}([10e-5, .9, .05]) for _ in 1:p])

DCC{1, 1, GARCH{1, 1}} specification.

───────────────────────
               β₁    α₁
───────────────────────
Parameters:  0.96  0.02
───────────────────────


In [6]:
linear_loss_sim750 = [0.0 for i in 1:30]

@showprogress for i in 1:30
    sim_data750 = simulate(simulation_spec750, 1000; warmup = 100).data
    est_model750 = fit(DCC{1,1}, sim_data750, method = :linear)
    est_covs750 = covariances(est_model750)
    linear_loss_sim750[i] = mean([loss(real, est) for (real,est) in zip(real_covs750, est_covs750)])
end

npzwrite("montecarlo//linear_loss_sim750.npy", linear_loss_sim750)

Progress: 100%|█████████████████████████████████████████| Time: 1:13:01


In [7]:
analytical_loss_sim750 = [0.0 for i in 1:30]

@showprogress for i in 1:30
    sim_data750 = simulate(simulation_spec750, 1000; warmup = 100).data
    est_model750 = fit(DCC{1,1}, sim_data750, method = :largescale)
    est_covs750 = covariances(est_model750)
    analytical_loss_sim750[i] = mean([loss(real, est) for (real,est) in zip(real_covs750, est_covs750)])
end

npzwrite("montecarlo//anal_loss_sim750.npy", analytical_loss_sim750)

Progress: 100%|█████████████████████████████████████████| Time: 1:09:38


In [6]:
sample_cov_sim750 = [0.0 for i in 1:30]

@showprogress for i in 1:30
    sim_data750 = simulate(simulation_spec750, 1000; warmup = 100).data
    est_model750 = fit(DCC{1,1}, sim_data750, method = :twostep)
    est_covs750 = covariances(est_model750)
    sample_cov_sim750[i] = mean([loss(real, est) for (real,est) in zip(real_covs750, est_covs750)])
end

npzwrite("montecarlo//sample_loss_sim750.npy", sample_cov_sim750)

Progress: 100%|█████████████████████████████████████████| Time: 1:07:17


### Monte Carlo 1000

In [39]:
mod1000 = fit(DCC{1,1}, data1000_mat)
p = 1000
R1000 = mod1000.spec.R;
real_covs1000 = covariances(mod1000)
simulation_spec1000 = DCC{1, 1}(R1000, [.96, .02], [GARCH{1, 1}([10e-5, .9, .05]) for _ in 1:p])

DCC{1, 1, GARCH{1, 1}} specification.

───────────────────────
               β₁    α₁
───────────────────────
Parameters:  0.96  0.02
───────────────────────


In [10]:
linear_loss_sim1000 = [0.0 for i in 1:20]

@showprogress for i in 1:20
    sim_data1000 = simulate(simulation_spec1000, 1250; warmup = 100).data
    est_model1000 = fit(DCC{1,1}, sim_data1000, method = :linear)
    est_covs1000 = covariances(est_model1000)
    linear_loss_sim1000[i] = mean([loss(real, est) for (real,est) in zip(real_covs1000, est_covs1000)])
end

npzwrite("montecarlo_new//linear_loss_sim1000.npy", linear_loss_sim1000)

Progress: 100%|█████████████████████████████████████████| Time: 3:05:05


In [6]:
analytical_loss_sim1000 = [0.0 for i in 1:20]

@showprogress for i in 1:20
    sim_data1000 = simulate(simulation_spec1000, 1250; warmup = 100).data
    est_model1000 = fit(DCC{1,1}, sim_data1000, method = :largescale)
    est_covs1000 = covariances(est_model1000)
    analytical_loss_sim1000[i] = mean([loss(real, est) for (real,est) in zip(real_covs1000, est_covs1000)])
end

npzwrite("montecarlo_new//anal_loss_sim1000.npy", analytical_loss_sim1000)

Progress: 100%|█████████████████████████████████████████| Time: 2:51:55


In [7]:
sample_cov_sim1000 = [0.0 for i in 1:20]

@showprogress for i in 1:20
    sim_data1000 = simulate(simulation_spec1000, 1250; warmup = 100).data
    est_model1000 = fit(DCC{1,1}, sim_data1000, method = :twostep)
    est_covs1000 = covariances(est_model1000)
    sample_cov_sim1000[i] = mean([loss(real, est) for (real,est) in zip(real_covs1000, est_covs1000)])
end

npzwrite("montecarlo_new//sample_loss_sim1000.npy", sample_cov_sim1000)

Progress: 100%|█████████████████████████████████████████| Time: 3:02:25


# Spectrum

In [15]:
function vector_of_matrices_to_3d_array(matrices::Vector{Matrix{Float64}})
    # Check if the input vector is empty
    if isempty(matrices)
        throw(ArgumentError("The vector of matrices is empty."))
    end

    # Get the dimensions of the first matrix
    rows, cols = size(matrices[1])

    # Ensure all matrices have the same dimensions
    for mat in matrices
        if size(mat) != (rows, cols)
            throw(ArgumentError("All matrices must have the same dimensions."))
        end
    end

    # Flatten the vector of matrices into a 2D array
    flattened = hcat(matrices...)

    # Reshape the flattened array into a 3D array
    array_3d = reshape(flattened, rows, cols, length(matrices))

    return array_3d
end

vector_of_matrices_to_3d_array (generic function with 1 method)

In [16]:
npzwrite("dcc_res_1000//dcc1000.npy", vector_of_matrices_to_3d_array(covariances(mod1000)))

In [43]:
npzwrite("dcc_res_1000//dcc_corr1000.npy", vector_of_matrices_to_3d_array(correlations(mod1000)))

### Long History

In [11]:
mod750long = fit(DCC{1,1}, data750_long_mat)
#npzwrite("dcc_res_long//dcc750long.npy", vector_of_matrices_to_3d_array(covariances(mod750long)))


750-dimensional DCC{1, 1} - GARCH{1, 1} - Intercept{Float64} specification, T=4948.

DCC parameters, estimated by largescale procedure:
─────────────────────
        β₁         α₁
─────────────────────
  0.979799  0.0116638
─────────────────────

Calculating standard errors is expensive. To show them, use
`show(IOContext(stdout, :se=>true), <model>)`


In [12]:
npzwrite("dcc_res_long//dcc_corr750long.npy", vector_of_matrices_to_3d_array(correlations(mod750long)))

LoadError: UndefVarError: `vector_of_matrices_to_3d_array` not defined

## Markowitz Portfolio

In [ ]:
data = data750_long_mat[end-1500:end,:]
start_index = 1
end_index = 1000
(n, p) = size(data)
list_of_portfolios = [Vector{Float64}(undef, p) for _ in 1:(n-1000)]

@showprogress for i in 1:(n-1000)
    list_of_portfolios[i] .= markowitz_min_var_port(predict(fit(DCC{1,1}, data[start_index:end_index, :])))
    start_index += 1
    end_index += 1
end

In [ ]:
npzwrite("markowitz_analytical.npy", list_of_portfolios)

In [26]:
data = data100_long_mat
start_index = 1
end_index = 500
(n, p) = size(data)
list_of_portfolios = [Vector{Float64}(undef, p) for _ in 1:(n-500)]

@showprogress for i in 1:(n-500)
    list_of_portfolios[i] .= markowitz_min_var_port(predict(fit(DCC{1,1}, data[start_index:end_index, :])))
    start_index += 1
    end_index += 1
end

Progress:   0%|█                                        |  ETA: 0:47:05┌ Warning: Failed to achieve finite new evaluation point, using alpha=0
└ @ LineSearches C:\Users\guibi\.julia\packages\LineSearches\jgnxK\src\hagerzhang.jl:155
Progress:   0%|█                                        |  ETA: 0:48:39┌ Warning: Failed to achieve finite new evaluation point, using alpha=0
└ @ LineSearches C:\Users\guibi\.julia\packages\LineSearches\jgnxK\src\hagerzhang.jl:155
Progress:   0%|█                                        |  ETA: 0:45:24┌ Warning: Failed to achieve finite new evaluation point, using alpha=0
└ @ LineSearches C:\Users\guibi\.julia\packages\LineSearches\jgnxK\src\hagerzhang.jl:155
Progress:   1%|█                                        |  ETA: 0:45:38┌ Warning: Failed to achieve finite new evaluation point, using alpha=0
└ @ LineSearches C:\Users\guibi\.julia\packages\LineSearches\jgnxK\src\hagerzhang.jl:155
Progress:   1%|█                                        |  ETA: 0:45:01┌

In [27]:
npzwrite("min_var_analysis//analytical100.npy",hcat(list_of_portfolios...)')

In [21]:
data = data100_long_mat
start_index = 1
end_index = 500
(n, p) = size(data)
list_of_portfolios_linear = [Vector{Float64}(undef, p) for _ in 1:(n-500)]

@showprogress for i in 1:(n-500)
    list_of_portfolios_linear[i] .= markowitz_min_var_port(predict(fit(DCC{1,1}, data[start_index:end_index, :], method = :linear)))
    start_index += 1
    end_index += 1
end

Progress:   0%|█                                        |  ETA: 0:42:21┌ Warning: Failed to achieve finite new evaluation point, using alpha=0
└ @ LineSearches C:\Users\guibi\.julia\packages\LineSearches\jgnxK\src\hagerzhang.jl:155
Progress:   0%|█                                        |  ETA: 0:42:17┌ Warning: Failed to achieve finite new evaluation point, using alpha=0
└ @ LineSearches C:\Users\guibi\.julia\packages\LineSearches\jgnxK\src\hagerzhang.jl:155
Progress:   1%|█                                        |  ETA: 0:42:14┌ Warning: Failed to achieve finite new evaluation point, using alpha=0
└ @ LineSearches C:\Users\guibi\.julia\packages\LineSearches\jgnxK\src\hagerzhang.jl:155
Progress:   1%|█                                        |  ETA: 0:42:00┌ Warning: Failed to achieve finite new evaluation point, using alpha=0
└ @ LineSearches C:\Users\guibi\.julia\packages\LineSearches\jgnxK\src\hagerzhang.jl:155
Progress:   1%|█                                        |  ETA: 0:41:44┌

In [27]:
npzwrite("min_var_analysis//linear100.npy",hcat(list_of_portfolios_linear...)')

### Analytical Markowitz

In [5]:
analytical_res = fit(DCC{1,1}, data750_long_mat)
covs_analytical = covariances(analytical_res)
analytical_ports = markowitz_min_var_port.(covs_analytical)
npzwrite("min_var_analysis//analytical.npy", hcat(analytical_ports...))

┌ Warning: Failed to achieve finite new evaluation point, using alpha=0
└ @ LineSearches C:\Users\guibi\.julia\packages\LineSearches\jgnxK\src\hagerzhang.jl:155


4780-element Vector{Vector{Float64}}:
 [-0.00940625309649501, -0.0015958721341972468, -2.093652662994657e-5, -0.004118217748079252, -0.004151310011976138, 0.00034708452316859444, -0.0022376756986507637, 0.008157249664921196, -0.0012563482213529414, 0.011333216788978296  …  -0.000233454932240332, -0.01305351972685805, 0.011202728605138431, -0.0012199969866098367, 8.116176630525576e-5, 0.0045148840850615605, -0.0014695507225800065, 0.0009616900091691392, -0.003734088937724561, 0.014413933206685214]
 [-0.009557454044554521, -0.0036194424177414353, 0.0012749023041546076, -0.002309027230260223, -0.002034138152444336, 0.00011082929511360301, -0.005240179335876748, 0.010013111560795382, -0.0016690437709132747, 0.011144833591379383  …  -0.0025405134408660723, -0.013896830208828488, 0.011464324181623842, -0.0011812220420636234, 9.676048357584616e-5, 0.004945536600581731, -0.0018869208143403777, 0.0007721843967439896, -0.003845531607655854, 0.014438061615826645]
 [-0.009282292289170647, -0.00431

In [12]:
linear_res = fit(DCC{1,1}, data750_long_mat,  method = :linear)
covs_linear = covariances(linear_res)
linear_ports = markowitz_min_var_port.(covs_linear)
npzwrite("min_var_analysis//linear.npy", hcat(linear_ports...))

┌ Warning: Failed to achieve finite new evaluation point, using alpha=0
└ @ LineSearches C:\Users\guibi\.julia\packages\LineSearches\jgnxK\src\hagerzhang.jl:155


In [13]:
sample_res = fit(DCC{1,1}, data750_long_mat,  method = :twostep)
covs_sample = covariances(sample_res)
sample_ports = markowitz_min_var_port.(covs_sample)
npzwrite("min_var_analysis//sample.npy", hcat(sample_ports...))

┌ Warning: Failed to achieve finite new evaluation point, using alpha=0
└ @ LineSearches C:\Users\guibi\.julia\packages\LineSearches\jgnxK\src\hagerzhang.jl:155
